#### Identifying bird species infected with H5N1 and finding bird density from past 30 days in Chicago (as of 7/15/2025).   
By: Angel Moreno

In [264]:
import pandas as pd
import geopandas as gpd
import os
import requests
from datetime import datetime
import matplotlib.pyplot as plt
import imageio.v2 as imageio
import numpy as np
from shapely import Point
# from dotenv import load_dotenv

#### Bird species infected with H5N1 in Midwestern states of interest:

In [243]:
df = pd.read_csv("HPAI Detections in Wild Birds.csv")

mw_states_of_interest = ["Illinois", "Wisconsin", "Michigan"]
dead_birds = df[(df["HPAI Strain"].str.contains("H5N1")) & (df["State"].isin(mw_states_of_interest)) & (df["Sampling Method"]=="Morbidity/Mortality")]
dead_birds["Bird Species"].unique()

array(['Bald eagle', 'Peregrine falcon', 'Sandhill crane',
       'Red-tailed hawk', 'Herring gull', 'Red-breasted merganser',
       'Great horned owl', 'Hawk (unidentified)', 'American crow',
       'Canada goose', 'Trumpeter swan', 'Bufflehead', 'Snowy owl',
       'Tundra swan', 'Chilean flamingo', 'Barred owl', 'Mallard',
       'Hooded merganser', 'Common loon', 'European starling',
       'Snow goose', 'Canvasback', 'Crow (unidentified)', "Ross's goose",
       'American white pelican', 'Common tern', 'Swan (unidentified)',
       'Blue-winged teal', 'Goose (unidentified)', 'Great blue heron',
       'Great egret', 'Pelican (unidentified)', 'Ring-billed gull',
       'Caspian tern', 'Double-crested cormorant', 'Wood duck',
       'Redhead duck', 'Red-winged blackbird', 'Gadwall',
       'Turkey vulture', 'Common raven', 'Ruddy duck',
       'Eagle (unidentified)', 'Rough-legged hawk', 'Red-shouldered hawk',
       "Cooper's hawk", 'Wood Duck', 'Mute swan',
       'Cormorant (uni

In [244]:
mw_states_of_interest = ["Illinois", "Wisconsin", "Michigan"]
dead_birds_il = df[(df["HPAI Strain"].str.contains("H5N1")) & (df["State"] == "Illinois") & (df["Sampling Method"]=="Morbidity/Mortality")]
dead_birds_il["Bird Species"].unique()

array(['Chilean flamingo', 'Red-breasted merganser', 'Bufflehead',
       'Canada goose', 'Mallard', 'Hooded merganser', 'Sandhill crane',
       'Snow goose', 'Crow (unidentified)', 'Bald eagle',
       'Red-tailed hawk', 'Great horned owl', "Ross's goose",
       'American white pelican', 'Common tern', 'Goose (unidentified)',
       'Swan (unidentified)', 'Pelican (unidentified)',
       'Double-crested cormorant', 'Turkey vulture',
       'Cormorant (unidentified)'], dtype=object)

#### Chicago Bird Migration 30-Day Data (eBird):

In [245]:
api_key = os.getenv("EBIRD_API_KEY")
headers = {"X-eBirdApiToken": api_key}

params = {
    "lat": 41.8781,
    "lng": -87.6298,
    "dist": 50, # 50km radius (will filter with shp file later)
    "back": 30, # past 30 days
    "maxResults": 10000
}

response = requests.get(
    "https://api.ebird.org/v2/data/obs/geo/recent",
    headers=headers,
    params=params
)

data = response.json()

In [246]:
obs_30d = pd.DataFrame(data)
obs_30d.shape

(195, 14)

In [247]:
chicago_boundary = gpd.read_file("Chicago_Tracts_2020.zip")

# Make sure both GeoDataFrames use the same CRS
chicago_boundary = chicago_boundary.to_crs("EPSG:4326")

# Convert eBird DataFrame to GeoDataFrame
geometry = [Point(xy) for xy in zip(obs_30d['lng'], obs_30d['lat'])]
gdf = gpd.GeoDataFrame(obs_30d, geometry=geometry, crs="EPSG:4326")

# Use unary_union to create a single polygon from tracts
chicago_union = chicago_boundary.union_all()

# Filter: keep only points within the city boundary
gdf_chicago = gdf[gdf.within(chicago_union)]

gdf_chicago

,speciesCode,comName,sciName,locId,locName,obsDt,howMany,lat,lng,obsValid,obsReviewed,locationPrivate,subId,exoticCategory,geometry
2,cangoo,Canada Goose,Branta canadensis,L294426,"Northerly Island, Chicago",2025-07-15 18:10,46,41.859614,-87.608568,True,False,False,S260194713,NaN,POINT (-87.60857 41.85961)
3,mallar3,Mallard,Anas platyrhynchos,L294426,"Northerly Island, Chicago",2025-07-15 18:10,6,41.859614,-87.608568,True,False,False,S260194713,NaN,POINT (-87.60857 41.85961)
4,killde,Killdeer,Charadrius vociferus,L294426,"Northerly Island, Chicago",2025-07-15 18:10,2,41.859614,-87.608568,True,False,False,S260194713,NaN,POINT (-87.60857 41.85961)
5,sposan,Spotted Sandpiper,Actitis macularius,L294426,"Northerly Island, Chicago",2025-07-15 18:10,4,41.859614,-87.608568,True,False,False,S260194713,NaN,POINT (-87.60857 41.85961)
6,ribgul,Ring-billed Gull,Larus delawarensis,L294426,"Northerly Island, Chicago",2025-07-15 18:10,4,41.859614,-87.608568,True,False,False,S260194713,NaN,POINT (-87.60857 41.85961)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,comloo,Common Loon,Gavia immer,L383782,"Rainbow Beach, Chicago",2025-06-24 15:30,1,41.758805,-87.546078,True,True,False,S253794481,NaN,POINT (-87.54608 41.75881)
187,semplo,Semipalmated Plover,Charadrius semipalmatus,L161180,"Montrose Point Bird Sanctuary, Lincoln Park, C...",2025-06-24 06:48,1,41.963383,-87.634420,True,True,False,S253706710,NaN,POINT (-87.63442 41.96338)
188,prawar,Prairie Warbler,Setophaga discolor,L161180,"Montrose Point Bird Sanctuary, Lincoln Park, C...",2025-06-23 20:14,1,41.963383,-87.634420,True,True,False,S253572510,NaN,POINT (-87.63442 41.96338)
193,yebfly,Yellow-bellied Flycatcher,Empidonax flaviventris,L161180,"Montrose Point Bird Sanctuary, Lincoln Park, C...",2025-06-18 08:20,1,41.963383,-87.634420,True,True,False,S251951963,NaN,POINT (-87.63442 41.96338)


In [248]:
print(type(gdf_chicago))
print(type(chicago_boundary))

<class 'geopandas.geodataframe.GeoDataFrame'>
<class 'geopandas.geodataframe.GeoDataFrame'>


In [249]:
projected_crs = "EPSG:4326"
chicago_boundary = chicago_boundary.to_crs(projected_crs)
gdf_chicago = gdf_chicago.to_crs(projected_crs)

date_format = "%Y-%m-%d %H:%M"

gdf_chicago["date"] = pd.to_datetime(gdf_chicago["obsDt"], format='ISO8601')

# will be used for animation
unique_dates = np.unique(gdf_chicago['date'])
len(unique_dates)

37

In [266]:
for day in unique_dates:
    fig, ax = plt.subplots(figsize=(8, 8))
    chicago_boundary.plot(ax=ax, color='lightgrey', edgecolor='black', alpha=0.1, zorder=1)

    plt.grid(color='gray', linestyle='--', linewidth=0.5)
    
    # day = day.reset_index(drop=True) # to prevent keyerror
    title = f"Bird Activity on {str(day)[0:10]}"
    ax.set_title(title, fontsize=14)

    var_to_plot = "howMany"

    gdf_chicago[gdf_chicago["date"]==day].scatter(
        column=var_to_plot,
        cmap="RdPu",
        legend=True,
        ax=ax,
        edgecolor="black",
        linewidth=0.2
    )

    folder_name = os.path.join("weather_var_maps")
    os.makedirs(folder_name, exist_ok=True)
    path = os.path.join(folder_name, f"{title}.png")

    plt.savefig(path)
    plt.close(fig)

In [268]:
images = []
frame_files = sorted(os.listdir("weather_var_maps"))
for file in frame_files:
    if file.endswith(".png"):
        images.append(imageio.imread(os.path.join("weather_var_maps", file)))

imageio.mimsave("bird_activity_heatmap.gif", images, fps=2)